NB: The below code is run using Google Colab

- Restart runtime before running
- 



In [ ]:
# Clone Waymo tools/resources
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip

In [ ]:
# Use tensorflow ver. 1
%tensorflow_version 1.x

import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip3 install waymo-open-dataset
import os
import math
import numpy as np
import pandas as pd
import itertools

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [ ]:
# Mount Google Drive, which will prompt for authorisation
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
import math
from google.colab import files

# FUNCTION FOR CONVERSION OF DATA TO DF/CSV
def frame_to_dataframe(frame_data, frame_num):
  nltk_tokens = nltk.word_tokenize(str(frame_data));

  df = pd.DataFrame();

  for i in range(len(nltk_tokens)):
    if nltk_tokens[i] == 'laser_labels':
      values1 = [[ nltk_tokens[i], nltk_tokens[i+47], nltk_tokens[i+6], nltk_tokens[i+9], nltk_tokens[i+12], \
                nltk_tokens[i+15], nltk_tokens[i+18], nltk_tokens[i+21], nltk_tokens[i+24], nltk_tokens[i+30],\
                nltk_tokens[i+33], nltk_tokens[i+36], nltk_tokens[i+39], nltk_tokens[i+43] ]];
      df=df.append(values1, ignore_index=True);
    if nltk_tokens[i] == 'camera_labels':
      if nltk_tokens[i+5] == 'labels':
        values2 = [[ nltk_tokens[i]+nltk_tokens[i+4], nltk_tokens[i+28], nltk_tokens[i+11], nltk_tokens[i+14],\
                    'NaN', nltk_tokens[i+17], nltk_tokens[i+20], 'NaN', 'NaN', 'NaN', 'NaN','NaN', 'NaN',\
                    nltk_tokens[i+24] ]];
        df=df.append(values2, ignore_index=True);
      elif nltk_tokens[i+5] == '}':
        continue
      for m in range(1,math.floor((len(nltk_tokens)-i)/31)):
        if nltk_tokens[i+5+26*m] != 'labels':
          break
        elif nltk_tokens[i+5+26*m] == 'labels':
          j = (i+5+26*m);
          values3 = [[ nltk_tokens[i]+nltk_tokens[i+4], nltk_tokens[j+23], nltk_tokens[j+6], nltk_tokens[j+9],\
                'NaN', nltk_tokens[j+12], nltk_tokens[j+15], 'NaN', 'NaN', 'NaN', 'NaN','NaN', 'NaN',\
                nltk_tokens[j+19] ]];
          df=df.append(values3, ignore_index=True);

  df.columns = ['device','iD', 'center_x', 'center_y', 'center_z', 'width', 'length', 'height', \
                'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y', 'type'];

  df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']]= \
   df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']].astype('float64')
  
  return df
  
  #frame_xlsx ='frame' + str(frame_num) + '.xlsx';
  #df.to_excel(r'/content/Extracted_Data/' + frame_xlsx);  #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html 
  #files.download('/content/Extracted_Data/' + frame_xlsx)

# FUNCTION FOR CALCULATION OF TRAFFIC USING DF
def DF_to_finaldata(df1, df2, final_data_append):
  # Determine distance moved by stationary sign in 2 frames
  is_sign = df1['type']=='TYPE_SIGN' # Filtering for stationary sign object
  df_sign = df1[is_sign] 
  AV_dist_x_1 = df_sign['center_x'].iloc[0] # Select stationary sign object in first row
  AV_dist_y_1 = df_sign['center_y'].iloc[0] # Take corresponding location values

  #find_speed = df2['iD'] == df_sign._get_value(0, 'iD') # NEED TO UNDERSTAND WHY THIS DOESNT WORK!! (for frame 1 but not frame 150)
  #https://www.kite.com/python/answers/how-to-get-a-value-from-a-cell-of-a-pandas-dataframe-in-python
  find_speed = df2['iD'] == df_sign.iloc[0]["iD"] # Go to next DF and find corresponding statiionary sign object
  df_sign_2 = df2[find_speed]

  AV_dist_x_2 = df_sign_2['center_x'].iloc[0] # metres
  AV_dist_y_2 = df_sign_2['center_y'].iloc[0] # Take corresponding location values; metres
  print(AV_dist_x_1, AV_dist_y_1, AV_dist_x_2, AV_dist_y_2)

  # Determine AV speed
  AV_speed_x = -(AV_dist_x_2 - AV_dist_x_1)/0.1 # m/s
  AV_speed_y = -(AV_dist_y_2 - AV_dist_y_1)/0.1 # m/s
 
  # Determine x-distance travelled by AV
  AV_dist_x = -(AV_dist_x_2 - AV_dist_x_1) # metres

  print(round(AV_speed_x,5), round(AV_speed_y, 5))

  df_AV = {'device': 'laser_labels', 'iD': 'AV', 'center_x':0, 'center_y':0, 'center_z':0, 'width':np.NaN, 'length':5.17, 'height':np.NaN, 'heading':0, 'speed_x':AV_speed_x, 'speed_y':AV_speed_y, 'accel_x':np.NaN, 'accel_y':np.NaN, 'type': 'TYPE_VEHICLE'} 
  # https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/

  df2 = df2.append(df_AV, ignore_index=True) # Append AV vehicle data to df2 (i.e. Frame 2)

  is_veh = df2['type']=='TYPE_VEHICLE' # Filter DF for vehicle objects
  df_veh = df2[is_veh]

  is_veh = df_veh['device']=='laser_labels' # Filter DF for lidar/laser data
  df_veh = df_veh[is_veh] 

  df_veh = df_veh.sort_values(by=['center_x']) # Sort DF by center_x values

  is_road = abs(df_veh['center_y']) <= 3.7/2 # Filter DF for vehicles on straight road
  df_road = df_veh[is_road]
  df_road

  no_turn = abs(df_road['heading']) <= math.pi/4 # Filter DF based on heading criterion
  df_straight = df_road[no_turn]
  df_straight

  # Calculation of inputs
  df_1 = [] 
  for index, row in df_straight.iterrows(): # https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
    x_b = row['center_x'] - row['length']*0.5*math.cos(row['heading']) # rear bumper coordinates
    y_b = row['center_y'] - row['length']*0.5*math.sin(row['heading'])

    x_f = row['center_x'] + row['length']*0.5*math.cos(row['heading']) # front bumper coordinates
    y_f = row['center_y'] + row['length']*0.5*math.sin(row['heading'])
 
    iD = row['iD']
    speed_tot = math.sqrt(row['speed_x']**2+row['speed_y']**2) # defining total speed value
    center_x = row['center_x']
    length = row['length']

    df_1.append([iD, speed_tot, center_x, length, x_b, y_b, x_f, y_f])

  df_2 = pd.DataFrame(df_1, columns=['iD', 'speed_tot', 'center_x', 'length','x_b', 'y_b', 'x_f', 'y_f'])

  # Shifting of rows, to allow for calculation between 2 consecutive rows
  df_2['shifted_x_b'] = df_2['x_b'].shift(-1) # https://stackoverflow.com/questions/42664418/referencing-to-the-next-index-in-iterrows
  df_2['shifted_y_b'] = df_2['y_b'].shift(-1)
  df_2['shifted_x_f'] = df_2['x_f'].shift(-1)
  df_2['shifted_y_f'] = df_2['y_f'].shift(-1)
  df_2

  # Calculation of traffic
  df_3 = []
  for index, row in df_2.iterrows():
    s = math.sqrt((row['shifted_x_b'] - row['x_f']) **2 + (row['shifted_y_b'] - row['y_f']) **2) # spacing (rear bumper to front bumper)
    f = math.sqrt((row['shifted_x_f'] - row['x_f']) **2 + (row['shifted_y_f'] - row['y_f']) **2) # front bumper to front bumper 
    h = f / row['speed_tot'] # headway
    df_3.append([s,f,h])

  df_3 = pd.DataFrame(df_3, columns=['s', 'f', 'h'])
  df_3

  df_4 = pd.concat([df_2, df_3], axis=1)
  df_4

  df_5 = df_4.drop(columns=['shifted_x_b', 'shifted_y_b', 'shifted_x_f', 'shifted_y_f']) # removed shifted columns
  df_5

  q_flow =  len(df_5) / df_5['h'].sum() *3600 # veh/hr/lane
  k_density = len(df_5) / df_5['s'].sum()  *1000 # veh/km/lane
  v_avgspeed = df_5['speed_tot'].mean()*3.6 # km/h
  x = AV_dist_x # metres
  d_framelenx = (df_5['center_x'].max() + df_5['length'][df_5['center_x'].idxmax()]*0.5)- \
    (df_5['center_x'].min() - df_5['length'][df_5['center_x'].idxmin()]*0.5) # Calculating distance between front and last vehicle
    # https://stackoverflow.com/questions/50464241/returning-the-actual-index-value-of-max-min-values-from-a-pandas-dataframe-col

  final_data = {'q': [q_flow], 'k': [k_density], 'v':[v_avgspeed], 'd': [d_framelenx], 'x': [x]}
  final_data = pd.DataFrame(final_data, columns = ['q', 'k', 'v', 'd', 'x'])

  final_data_append = final_data_append.append(final_data, ignore_index = True)
  return final_data_append

  #final_data_xlsx ='final_data' + str(frame_num) + '.xlsx';
  #final_data_append.to_excel(r'/content/Extracted_Data/' + final_data_xlsx);
  #files.download('/content/Extracted_Data/' + final_data_xlsx)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
FILENAME = '/gdrive/My Drive/Colab Notebooks/training_0001.tar/segment-10599748131695282446_1380_000_1400_000_with_camera_labels.tfrecord'
# OR FILENAME = '/content/waymo-od/tutorial/frames'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')


i = 0 # initialise no. of frames as 0
i2 = 1 # initialise
j = 1 # start frame
k = 198 # end frame
d_append = pd.DataFrame()
for data in dataset:

  i += 1 # count number of frames; where i_max = 200 frames (or 200*5 = 1000 images altogether)

  if i<=j-1:
    continue
  frame = open_dataset.Frame() # type(frame) = waymo_open_dataset.dataset_pb2.Frame
  frame.ParseFromString(bytearray(data.numpy()))

  (range_images, camera_projections,
  range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(
  frame)

  if i2%2 != 0: # if index is odd
    for index, image in enumerate(frame.images):
      df1 = frame_to_dataframe(frame, i)

  if i2 %2 == 0: # if index is even
  # https://beginnersbook.com/2018/01/python-continue-statement/
    for index, image in enumerate(frame.images):
      df2 = frame_to_dataframe(frame, i)
      if (index % 5) == 0: # https://stackoverflow.com/questions/57546406/how-to-skip-few-iterations-in-a-for-loop-based-on-a-if-condition
        d_append = DF_to_finaldata(df1, df2, d_append) # Profita
        final_data_xlsx ='final_data' + str(i) + '.xlsx';
        d_append.to_excel(r'/content/Extracted_Data/' + final_data_xlsx);
      else:
        pass
    df1=df2
    i2 += 1

  i2 += 1

  if i==k: 
    break

#final_data_xlsx ='final_data' + str(i) + '.xlsx';
#d_append.to_excel(r'/content/Extracted_Data/' + final_data_xlsx);
!zip -r /content/Extracted_Data/Extracted_Data.zip /content/Extracted_Data # Create zip folder, and list location of its contents you want to zip
from google.colab import files
files.download('/content/Extracted_Data/Extracted_Data.zip')

In [ ]:
# https://medium.com/p/717b88a128c0/responses/show
#ctrl+shift+i to open console; paste below in console
#function ConnectButton(){
#    console.log("Connect pushed")
#    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
#}
#setInterval(ConnectButton,60000)